In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import os

In [ ]:
# read csv file
ventas = pd.read_csv(filepath_or_buffer= "/Users/raul/Documents/Isaac/uteq/3ro/bd/Ventas.csv")

In [ ]:
#Seleccion de datos relevantes para analisis de ventas por sucursal por año
sucVentas = ventas.loc[:, ['id_Sucursal', 'FechaPago', 'Monto_total']]

In [ ]:
sucVentas.dtypes

In [ ]:
# Correcion de errores estructurales
sucVentas['Monto_total'] = sucVentas['Monto_total'].str.replace('$', '').str.replace(',', '').astype(float)
sucVentas['FechaPago'] = pd.to_datetime(sucVentas['FechaPago'], format='%d/%m/%Y')
sucVentas['anio'] = sucVentas['FechaPago'].dt.year

In [ ]:
sucVentas.dtypes

In [ ]:
# Diagrama de caja de la columna "Monto_total"
plt.figure(figsize=(8, 6))
plt.boxplot(sucVentas['Monto_total'])
plt.title('Diagrama de Caja de Monto_total')
plt.show()

In [ ]:
# Gráfico de dispersión de la columna "Monto_total" vs "id_Sucursal"
plt.scatter(sucVentas['id_Sucursal'], sucVentas['Monto_total'])
plt.xlabel('Días')
plt.ylabel('Monto_total')
plt.title('Gráfico de Dispersión')
plt.show()

In [ ]:
# Histograma de la columna "Monto_total"
sns.histplot(sucVentas['Monto_total'], kde=True)
plt.xlabel('Monto_total')
plt.title('Histograma con Densidad de Monto_total')
plt.show()

In [ ]:
datos_faltantes = sucVentas.isnull().sum()
print(datos_faltantes)

In [ ]:
duplicados = sucVentas.duplicated()
print(duplicados)

In [ ]:
año_actual = 2023  # Año actual
año_siguiente = 2024  # Año siguiente
datos_año_actual = sucVentas[sucVentas['anio'] == año_actual]
datos_año_siguiente = sucVentas[sucVentas['anio'] == año_siguiente]

In [ ]:
datos_agrupados = datos_año_actual.groupby(['id_Sucursal', 'anio'])['Monto_total'].sum().reset_index()

In [ ]:
sucVentas.to_csv('sucVentas_preprocesados.csv')

In [ ]:
# Para asignar valores de columnas a los ejes
X = datos_agrupados[['id_Sucursal', 'anio']]
Y = datos_agrupados['Monto_total']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
modelo = LinearRegression()
modelo.fit(X_train, Y_train)

In [ ]:
X_prediccion = [[id_sucursal, año_siguiente] for id_sucursal in datos_agrupados['id_Sucursal'].unique()]
predicciones = modelo.predict(X_prediccion)

In [ ]:
predicciones_por_sucursal = pd.DataFrame({'id_Sucursal': datos_agrupados['id_Sucursal'].unique(), 'Prediccion': predicciones})
print(predicciones_por_sucursal)

In [ ]:
# Ventas por sucursal 2023
fig, ax = plt.subplots(figsize=(10, 6))

# Crear gráfico de barras agrupado por 'id_Sucursal'
datos_agrupados.pivot(index='anio', columns='id_Sucursal', values='Monto_total').plot(kind='bar', ax=ax)

# Personalizar el gráfico
ax.set_xlabel('Año')
ax.set_ylabel('Monto Total')
ax.set_title('Monto Total por Sucursal 2023')

# Mostrar el gráfico
plt.show()

In [ ]:
# Ventas por sucursal 2024
fig, ax = plt.subplots(figsize=(10, 6))

# Crear un gráfico de barras para las predicciones
ax.bar(predicciones_por_sucursal['id_Sucursal'], predicciones_por_sucursal['Prediccion'])

# Personalizar el gráfico
ax.set_xlabel('ID de Sucursal')
ax.set_ylabel('Monto Total Predicho')
ax.set_title('Predicciones de Ventas por Sucursal 2024')

# Mostrar el gráfico
plt.show()

In [ ]:
predicciones_por_sucursal.to_csv('predicciones_por_sucursal.csv')